In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e

fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-e option requires 1 argument


In [ ]:
from google.colab import userdata

In [ ]:
import wandb


In [ ]:
wandb.login(key=userdata.get('WANDB_API_KEY'))
hf_token = userdata.get('HF_TOKEN')
!huggingface-cli login --token {hf_token}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abdelati88 (abdelati88-nna-asia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `FT` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `FT`


In [ ]:
# مسار الداتا والدرايف بتاعك
data_dir = "/gdrive/MyDrive/finetune"

# الموديل اللي عندك محمّل في درايف
base_model_id = "/gdrive/MyDrive/finetune/mymodels/Qwen2.5-0.5B-Instruct"


In [ ]:
pip install json_repair

In [ ]:
!pip -q install -U "numpy>=2.0.0" \
                     "transformers==4.48.3" \
                     "accelerate>=0.34.2" \
                     "tokenizers>=0.20.3" \
                     "huggingface_hub>=0.25.2" \
                     "safetensors>=0.4.5" einops
# لو مش هتستخدم 4-bit سيب bitsandbytes. لو محتاجه:
# !pip -q install bitsandbytes==0.43.3


In [ ]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [ ]:
!pip install -U numpy
!pip install -U transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 103.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [ ]:
device = "cpu"
torch_dtype = None

def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

In [ ]:
story = """
ذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال،
 حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.

التقرير الذي يستند إلى أبحاث الأستاذ الجامعي شاين إنيت حول
الرفاه المالي يوضح أن لكل شخص "شخصية مالية" تتحدد وفقا لطريقة
 تفاعله مع المال، والتي تتأثر بشكل مباشر بتربية الأسرة وتجارب الطفولة.

 الأبعاد الثلاثة للعلاقة بالمال
بحسب الدراسة، هناك ثلاثة أبعاد رئيسية تشكّل علاقتنا بالمال:

الاكتساب (A): يميل الأفراد الذين ينتمون لهذا
 البعد إلى اعتبار المال سلعة قابلة للجمع، حيث يرون
في تحقيق الثروة هدفا بحد ذاته. والجانب السلبي لهذا
 النمط هو إمكانية التحول إلى هوس بالثروة أو العكس،
 أي رفض تام لاكتساب المال باعتباره مصدرا للفساد.

الاستخدام (U): يرى هؤلاء الأشخاص المال أداة للتمتع بالحياة، حيث يربطون قيمته بقدرته على توفير
المتعة والراحة. ومع ذلك، قد يصبح
البعض مدمنا على الإنفاق، في حين يتجه آخرون إلى التقشف المفرط خوفا من المستقبل.

الإدارة (M): أصحاب هذا النمط يعتبرون المال مسؤولية تتطلب التخطيط الدقيق. لكن في بعض الحالات،
 قد يتحول الأمر إلى هوس مفرط بإدارة الإنفاق، مما يؤثر سلبا على العلاقات الشخصية.

 كيف تؤثر العائلة على علاقتنا بالمال؟
يشير التقرير إلى أن التجارب الأسرية تلعب دورا رئيسيا في تحديد
 "الشخصية المالية" لكل فرد، على سبيل المثال، إذا كان أحد الوالدين يعتمد على المال
كمكافأة للسلوك الجيد، فقد يتبنى الطفل لاحقا النمط نفسه في حياته البالغة.

لتحليل هذه التأثيرات بشكل دقيق، طورت رابطة العلاج المالي
(Financial Therapy Association) أداة تسمى مخطط الجينوم المالي (Money Genogram)،
وهو نموذج يُستخدم لتحديد الأنماط المالية داخل العائلة.

تتضمن هذه الأداة:

رسم شجرة عائلية.
تصنيف أفراد العائلة وفقا للأبعاد الثلاثة للعلاقة بالمال (A ،U ،M).
تحديد ما إذا كان السلوك المالي لكل فرد صحيا (+) أو غير صحي (-).
على سبيل المثال، إذا نشأ شخص في عائلة
اعتادت على الإنفاق المفرط، فقد يكون لديه ميل قوي إلى اتباع النمط نفسه،
 أو العكس تماما، حيث يصبح مقتصدا بشكل مبالغ فيه كرد فعل نفسي.
"""

In [ ]:
StoryCategory = Literal[
    "politics", "sports", "art", "technology", "economy",
    "health", "entertainment", "science",
    "not_specified"
]

EntityType = Literal[
    "person-male", "person-female", "location", "organization", "event", "time",
    "quantity", "money", "product", "law", "disease", "artifact", "not_specified"
]

class Entity(BaseModel):
    entity_value: str = Field(..., description="The actual name or value of the entity.")
    entity_type: EntityType = Field(..., description="The type of recognized entity.")

class NewsDetails(BaseModel):
    story_title: str = Field(..., min_length=5, max_length=300,
                             description="A fully informative and SEO optimized title of the story.")

    story_keywords: List[str] = Field(..., min_items=1,
                                      description="Relevant keywords associated with the story.")

    story_summary: List[str] = Field(
                                    ..., min_items=1, max_items=5,
                                    description="Summarized key points about the story (1-5 points)."
                                )

    story_category: StoryCategory = Field(..., description="Category of the news story.")

    story_entities: List[Entity] = Field(..., min_items=1, max_items=10,
                                        description="List of identified entities in the story.")


In [ ]:
details_extraction_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            json.dumps(
                NewsDetails.model_json_schema(), ensure_ascii=False
            ),
            "",

            "## Story Details:",
            "```json"
        ])
    }
]

In [ ]:
class TranslatedStory(BaseModel):
    translated_title: str = Field(..., min_length=5, max_length=300,
                                  description="Suggested translated title of the news story.")
    translated_content: str = Field(..., min_length=5,
                                    description="Translated content of the news story.")

targeted_lang = "English"

translation_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are a professional translator.",
            "You will be provided by an Arabic text.",
            "You have to translate the text into the `Targeted Language`.",
            "Follow the provided Scheme to generate a JSON",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content":  "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
            "",

            "## Targeted Language:",
            targeted_lang,
            "",

            "## Translated Story:",
            "```json"

        ])
    }
]

In [ ]:
from huggingface_hub import snapshot_download
target_dir = "/gdrive/MyDrive/finetune/mymodels/Qwen2.5-0.5B-Instruct"

_ = snapshot_download(
    repo_id="Qwen/Qwen2.5-0.5B-Instruct",
    local_dir=target_dir,
    local_dir_use_symlinks=False,
    token=None,  # أو خليها None لو الموديل public
)

base_model_id = target_dir

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch_dtype,
    trust_remote_code=True,
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

base_model_id = "/gdrive/MyDrive/finetune/mymodels/Qwen2.5-0.5B-Instruct"

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)

# تحديد نوع البيانات (float16 لو فيه GPU)
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# تحميل الموديل
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto" if torch.cuda.is_available() else None,
    torch_dtype=torch_dtype,   # <-- هنا استخدم torch_dtype بدل dtype
    trust_remote_code=True,
)


In [ ]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [ ]:
text = tokenizer.apply_chat_template(
    details_extraction_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
response

'{\n  "story_title": "التأثيرات الاجتماعية على العلاقة بالمال",\n  "story_keywords": [\n    "العادات المالية",\n    "ال familias",\n    "الرقم الإجمالي"\n  ],\n  "story_summary": "ذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال، حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.",\n  "story_category": "Economy",\n  "story_entities": [\n    {\n      "entity_value": "العادات المالية",\n      "entity_type": "Person-Male"\n    },\n    {\n      "entity_value": "العادات المالية",\n      "entity_type": "Person-Female"\n    }\n  ]\n}\n```'

In [ ]:
text = tokenizer.apply_chat_template(
    translation_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)

NameError: name 'response' is not defined

In [ ]:
from huggingface_hub import snapshot_download

target_dir = "/gdrive/MyDrive/finetune/mymodels/Qwen2.5-7B-Instruct"

_ = snapshot_download(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    local_dir=target_dir,
    local_dir_use_symlinks=False,
    token=None,  # خليها None لو الموديل public
)

print("✅ Model downloaded to:", target_dir)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

✅ Model downloaded to: /gdrive/MyDrive/finetune/mymodels/Qwen2.5-7B-Instruct


In [ ]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 7.3 MB/s eta 0:00:00


In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id_t = "/gdrive/MyDrive/finetune/mymodels/Qwen2.5-7B-Instruct"

# إعداد 4-bit quantization (يوفر VRAM)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# تحميل التوكنيزر
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)

# تحميل الموديل
model = AutoModelForCausalLM.from_pretrained(
    base_model_id_t,
    device_map="auto",     # يوزع الموديل على الـ GPU
    quantization_config=bnb_config,
    trust_remote_code=True,
)

print("✅ Model loaded successfully")

AttributeError: 'frozenset' object has no attribute 'discard'

In [ ]:
# 1) خُد الجهاز من الموديل الكبير
device = model.device  # cuda:0 غالباً

# 2) لو عندك الرسائل جاهزة (translation_messages) خليك عليها
# مثال للشكل الصحيح لو حابب تتأكد:
# translation_messages = [
#   {"role": "system", "content": "You are a helpful assistant that replies in Arabic."},
#   {"role": "user", "content": "ترجم للجملة التالية للإنجليزية: الذكاء الاصطناعي رائع."}
# ]

# 3) ابني النص بنمط Qwen chat template
text = tokenizer.apply_chat_template(
    translation_messages,
    tokenize=False,
    add_generation_prompt=True
)

# 4) ترميز الإدخال على نفس جهاز الموديل
inputs = tokenizer(text, return_tensors="pt").to(device)

# 5) التوليد (بدون sampling هنا؛ لو عايز عيّن غيّر do_sample=True وحدد temperature/top_p)
with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        temperature=None,
        top_p=None,
        top_k=None,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

# 6) قصّ الجزء المتولّد فقط (بعد طول الإدخال)
gen_ids = outputs[0, inputs["input_ids"].shape[1]:]
response = tokenizer.decode(gen_ids, skip_special_tokens=True)

print(response)


{
  "translated_title": "How Does Family Influence Our Relationship with Money?",
  "translated_content": "According to Forbes magazine, the family plays a crucial role in shaping individuals' relationship with money, where this relationship is influenced by inherited patterns of financial behavior across generations.\n\nThe report, based on research by university professor Shane Inge on financial well-being, highlights that every person has a 'financial personality' determined by their interaction with money, which is directly affected by family upbringing and childhood experiences.\n\nThree Dimensions of Money Relationship\nAccording to the study, there are three main dimensions that shape our relationship with money:\n\nAcquisition (A): Individuals who belong to this dimension tend to view money as a commodity to be accumulated, seeing wealth accumulation as an end in itself. The negative aspect of this pattern is the potential for becoming obsessed with wealth or its opposite, name

In [ ]:
device = "cpu"  # تغيير من "cuda" إلى "cpu"
torch_dtype = torch.float32  # استخدام دقة float32 مع CPU

# تحميل النموذج بدون bitsandbytes
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map=device,  # لا حاجة لاستخدام GPU
    torch_dtype=torch_dtype,  # دقة standard
    trust_remote_code=True
)

print("✅ Model loaded successfully")


✅ Model loaded successfully


In [ ]:
!pip install -U bitsandbytes
!pip install --upgrade numpy


In [ ]:
import json
import random
from os.path import join

# مسار الفولدر بتاعك في Google Drive
data_dir = "/gdrive/MyDrive/finetune"

# مسار ملف الداتا
raw_data_path = join(data_dir, "finetune","datasets", "news-sample.jsonl")

# تحميل البيانات
raw_data = []
with open(raw_data_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip() == "":
            continue
        raw_data.append(json.loads(line.strip()))

# Shuffle مع تثبيت البذرة (seed) عشان الترتيب يفضل ثابت
random.Random(101).shuffle(raw_data)

print(f"✅ Raw data loaded: {len(raw_data)} samples")


✅ Raw data loaded: 2400 samples


In [ ]:
raw_data[0]['content']

'يواصل المعهد العربي في باريس استقبال زواره في معرض ما تقدمه فلسطين للعالم لإطلاعهم على الإرث الثقافي والفني للفلسطينيين؛ من خلال أعمال فنية لآمالهم وصور لواقعهم الأليم تحت الاحتلال. \n ويرى رئيس المعهد جاك لانغ -الذي أُعيد انتخابه قبل أيام للدورة الرابعة- ما يحدث في غزة حاليا جراء العدوان الإسرائيلي أنه كارثة. \n والمعهد هو مركز ثقافي وواجهة دبلوماسية يديرها لانغ منذ 2013 ويقع على ضفة نهر السين في باريس. \n وأشار لانغ، الذي شغل سابقا منصب وزير الثقافة بفرنسا، إلى أن المعرض هو إهداء للشعب الفلسطيني، ومُدّد ليستقبل مزيدا من الزوار حتى 31 ديسمبركانون الأول الجاري. \n ويضم المعرض، الذي افتُتح أواخر مايوأيار الماضي، حسب لانغ العديد من المعارض الفرعية عن فلسطين وعن غزة بالتحديد، من بينها معرض الصور اليومية عن الحياة في غزة. \n كما يشتمل على معرض الصور الفوتوكرومية القائم على تلوين صور من فلسطين تعود للقرن الـ19. \n ويعرض الفنان الفلسطيني محمد أبو سل عملا فريدا بعنوان مترو غزة، وهو عبارة عن عمل تركيبي متعدد الوسائط، لاقى إعجابا من الزوار. \n ويحضر الشاعر الفلسطيني الراحل محمود درويش من خلال 

In [ ]:
import json, json_repair, re, os
from os.path import join
from tqdm.auto import tqdm
import torch

# مسارات
data_dir = "/gdrive/MyDrive/finetune"
save_to = join(data_dir, "finetune","datasets", "sft.jsonl")
os.makedirs(os.path.dirname(save_to), exist_ok=True)

# عدّادات (بديل لتكلفة OpenAI)
prompt_tokens = 0
completion_tokens = 0

# مساعد: اقتطاع JSON من نص حر
def extract_json(text: str):
    # جرّب بين ```json ... ``` أولاً
    m = re.search(r"```json\s*(\{.*\}|\[.*\])\s*```", text, flags=re.S)
    if not m:
        # fallback: أول شكل JSON ظاهر في النص
        m = re.search(r"(\{.*\}|\[.*\])", text, flags=re.S)
    if not m:
        return None
    raw = m.group(1)
    try:
        return json.loads(raw)
    except Exception:
        try:
            return json_repair.loads(raw)  # يصلّح JSON مكسور
        except Exception:
            return None

# مساعد: توليد من المعلّم
def ask_teacher(messages, max_new_tokens=512, temperature=0.2):
    # نصّ الـ chat بنمط Qwen
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    # ترميز وإرسال لنفس جهاز الموديل
    device = model.device
    inputs = tokenizer(text, return_tensors="pt").to(device)
    input_len = inputs["input_ids"].shape[1]

    with torch.inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=(temperature is not None and temperature > 0),
            temperature=temperature if temperature else None,
            top_p=0.9 if temperature else None,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    # الجزء المتولد فقط
    gen_ids = out[0, input_len:]
    reply = tokenizer.decode(gen_ids, skip_special_tokens=True)

    # حساب توكنز تقريبي
    in_tok = inputs["input_ids"].numel()
    out_tok = gen_ids.numel()
    return reply, in_tok, out_tok

# حلقة إنتاج الـSFT من المعلّم
ix = 0
# اكتب الهيدر لو حابب تبدأ بملف جديد (اختياري): open(save_to, "w").close()
for story in tqdm(raw_data):
    # نفس الـ messages اللي كنت بتبعتها لـ OpenAI
    sample_details_extraction_messages = [
        {
            "role": "system",
            "content": "\n".join([
                "You are an NLP data paraser.",
                "You will be provided by an Arabic text associated with a Pydantic scheme.",
                "Generate the output in the same story language.",
                "You have to extract JSON details from text according the Pydantic details.",
                "Extract details as mentioned in text.",
                "Do not generate any introduction or conclusion."
            ])
        },
        {
            "role": "user",
            "content": "\n".join([
                "## Story:",
                story['content'].strip(),
                "",
                "## Pydantic Details:",
                json.dumps(NewsDetails.model_json_schema(), ensure_ascii=False),
                "",
                "## Story Details:",
                "```json"
            ])
        }
    ]

    try:
        llm_response, in_tok, out_tok = ask_teacher(
            sample_details_extraction_messages,
            max_new_tokens=512,
            temperature=0.2
        )
    except RuntimeError as e:
        # لو حصل OOM قلّل max_new_tokens وحاول تاني
        if "out of memory" in str(e).lower():
            torch.cuda.empty_cache()
            llm_response, in_tok, out_tok = ask_teacher(
                sample_details_extraction_messages,
                max_new_tokens=256,
                temperature=0.2
            )
        else:
            continue

    llm_resp_dict = extract_json(llm_response)
    if not llm_resp_dict:
        continue

    with open(save_to, "a", encoding="utf-8") as f:
        f.write(json.dumps({
            "id": ix,
            "story": story['content'].strip(),
            "task": "Extract the story details into a JSON.",
            "output_scheme": json.dumps(NewsDetails.model_json_schema(), ensure_ascii=False),
            "response": llm_resp_dict
        }, ensure_ascii=False, default=str) + "\n")

    ix += 1
    prompt_tokens += in_tok
    completion_tokens += out_tok

    if (ix % 3) == 0:
        print(f"Iteration {ix}: prompt_tokens={prompt_tokens}, completion_tokens={completion_tokens}")

print(f"✅ Done. Written samples: {ix}")
print(f"📄 Saved to: {save_to}")


  0%|          | 0/2400 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import json
import random
from os.path import join

sft_data_path = join(data_dir, "finetune","datasets", "sft.jsonl")
llm_finetunning_data = []

system_message = "\n".join([
    "You are a professional NLP data parser.",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "Do not generate any introduction or conclusion."
])

# قراءة البيانات من السجل
for line in open(sft_data_path):
    if line.strip() == "":
        continue

    rec = json.loads(line.strip())

    llm_finetunning_data.append({
        "system": system_message,
        "instruction": "\n".join([
            "# Story:",
            rec["story"],  # القصّة

            "# Task:",
            rec["task"],  # المهمة التي سيتم القيام بها

            "# Output Scheme:",
            rec["output_scheme"],  # الصيغة التي يجب أن يتبعها الموديل
            "",

            "# Output JSON:",
            "```json"

        ]),
        "input": "",
        "output": "\n".join([
            "```json",
            json.dumps(rec["response"], ensure_ascii=False, default=str),  # الناتج الذي يطلبه الموديل
            "```"
        ]),
        "history": []  # سجل التاريخ للمهام السابقة
    })

# عشوائية البيانات
random.Random(101).shuffle(llm_finetunning_data)

# طباعة معلومات عن البيانات
print(f"Total data samples: {len(llm_finetunning_data)}")


Total data samples: 2766


In [ ]:
train_sample_sz = 2700

train_ds = llm_finetunning_data[:train_sample_sz]
eval_ds = llm_finetunning_data[train_sample_sz:]

os.makedirs(join(data_dir, "finetune","datasets", "llamafactory-finetune-data"), exist_ok=True)

with open(join(data_dir, "finetune","datasets", "llamafactory-finetune-data", "train.json"), "w") as dest:
    json.dump(train_ds, dest, ensure_ascii=False, default=str)

with open(join(data_dir,"finetune", "datasets", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    json.dump(eval_ds, dest, ensure_ascii=False, default=str)

In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: news_finetune_train
eval_dataset: news_finetune_val
template: qwen
cutoff_len: 3500
# max_samples: 50
overwrite_cache: true
preprocessing_num_workers: 16

### output
# resume_from_checkpoint: /gdrive/MyDrive/youtube-resources/llm-finetuning/models/checkpoint-1500
output_dir: /gdrive/MyDrive/finetune/finetune/llm-finetuning/models/
logging_steps: 10
save_steps: 500
plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100

report_to: wandb
run_name: newsx-finetune-llamafactory

push_to_hub: true
export_hub_model_id: "abdelati88/news-analyzer"
hub_private_repo: true
hub_strategy: checkpoint


Writing /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
!pip install -r requirements.txt


fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.
/content/LLaMA-Factory


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

/bin/bash: line 1: cd: LLaMA-Factory/: No such file or directory


In [ ]:
!ls /content/LLaMA-Factory/


assets	      docker	  LICENSE      pyproject.toml  requirements.txt  src
CITATION.cff  evaluation  Makefile     README.md       scripts		 tests
data	      examples	  MANIFEST.in  README_zh.md    setup.py


In [ ]:
!ls /content/LLaMA-Factory/examples/train_lora/


gpt_lora_sft.yaml	   llama3_lora_reward.yaml   llama4_lora_sft_ds3.yaml
llama3_lora_dpo.yaml	   llama3_lora_sft_ds3.yaml  news_finetune.yaml
llama3_lora_eval.yaml	   llama3_lora_sft_ray.yaml  qwen2_5vl_lora_dpo.yaml
llama3_lora_kto.yaml	   llama3_lora_sft.sh	     qwen2_5vl_lora_sft.yaml
llama3_lora_ppo.yaml	   llama3_lora_sft.yaml
llama3_lora_pretrain.yaml  llama3_preprocess.yaml


In [ ]:
%cd /content/LLaMA-Factory
!pip install -r requirements.txt


/content/LLaMA-Factory


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

/bin/bash: line 1: cd: LLaMA-Factory/: No such file or directory


In [ ]:
!llama_factory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


/bin/bash: line 1: llama_factory-cli: command not found


In [ ]:
!python3 /content/LLaMA-Factory/llama_factory_cli.py train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


python3: can't open file '/content/LLaMA-Factory/llama_factory_cli.py': [Errno 2] No such file or directory


In [ ]:
!pip install -r /content/LLaMA-Factory/requirements.txt


In [ ]:
!ls /content/LLaMA-Factory/


assets	      docker	  LICENSE      pyproject.toml  requirements.txt  src
CITATION.cff  evaluation  Makefile     README.md       scripts		 tests
data	      examples	  MANIFEST.in  README_zh.md    setup.py


In [ ]:
!find . -name "llama_factory_cli.py"


In [ ]:
!pip install transformers datasets wandb huggingface_hub


In [ ]:
!python3 /content/LLaMA-Factory/llama_factory_cli.py train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


python3: can't open file '/content/LLaMA-Factory/llama_factory_cli.py': [Errno 2] No such file or directory


In [ ]:
!pip install git+https://github.com/hiyouga/LLaMA-Factory.git


  Cloning https://github.com/hiyouga/LLaMA-Factory.git to /tmp/pip-req-build-5zn7ciz2
  Running command git clone --filter=blob:none --quiet https://github.com/hiyouga/LLaMA-Factory.git /tmp/pip-req-build-5zn7ciz2
  Resolved https://github.com/hiyouga/LLaMA-Factory.git to commit 2b27283ba0566eda9ec7ac335642807189c87e70
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.4.dev0-py3-none-any.whl size=313666 sha256=2ade8c04e46efeacf33c1bc33c7ce9d555305633235c67293dcb1af441247d91
  Stored in directory: /tmp/pip-ephem-wheel-cache-td4ulhrr/wheels/5d/78/2c/fd3cd7af2a43791f8ebc7faea54d7afab930a287c743453b2c
Successfully built llamafactory


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

/bin/bash: line 1: cd: LLaMA-Factory/: No such file or directory


In [ ]:
# حفظ ملف التعريف
!mkdir -p /content/LLaMA-Factory/examples/train_lora

# كتابة محتوى الملف news_finetune.yaml في مسار معين داخل Colab
news_finetune_yaml = """
### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: news_finetune_train
eval_dataset: news_finetune_val
template: qwen
cutoff_len: 3500
overwrite_cache: true
preprocessing_num_workers: 16

### output
output_dir: /content/drive/MyDrive/finetune/models/
logging_steps: 10
save_steps: 500
plot_loss: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100

report_to: wandb
run_name: newsx-finetune-llamafactory

push_to_hub: true
export_hub_model_id: "abdelati88/news-analyzer"
hub_private_repo: true
hub_strategy: checkpoint
"""

with open("/content/LLaMA-Factory/examples/train_lora/news_finetune.yaml", "w") as f:
    f.write(news_finetune_yaml)


In [ ]:
!cd /content/LLaMA-Factory && llama-factory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


/bin/bash: line 1: llama-factory-cli: command not found


In [ ]:
%cd /content/LLaMA-Factory


/content/LLaMA-Factory


In [ ]:
!pip install git+https://github.com/hiyouga/LLaMA-Factory.git


  Cloning https://github.com/hiyouga/LLaMA-Factory.git to /tmp/pip-req-build-4277butt
  Running command git clone --filter=blob:none --quiet https://github.com/hiyouga/LLaMA-Factory.git /tmp/pip-req-build-4277butt
  Resolved https://github.com/hiyouga/LLaMA-Factory.git to commit 2b27283ba0566eda9ec7ac335642807189c87e70
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
%cd /content/LLaMA-Factory
!pip install git+https://github.com/hiyouga/LLaMA-Factory.git


/content/LLaMA-Factory
  Cloning https://github.com/hiyouga/LLaMA-Factory.git to /tmp/pip-req-build-3f358zr8
  Running command git clone --filter=blob:none --quiet https://github.com/hiyouga/LLaMA-Factory.git /tmp/pip-req-build-3f358zr8
  Resolved https://github.com/hiyouga/LLaMA-Factory.git to commit 2b27283ba0566eda9ec7ac335642807189c87e70
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!cd /content/LLaMA-Factory && llama-factory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


/bin/bash: line 1: llama-factory-cli: command not found


In [ ]:
!pip install -r /content/LLaMA-Factory/requirements.txt



In [ ]:
!cd /content/LLaMA-Factory && llama-factory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


/bin/bash: line 1: llama-factory-cli: command not found


In [ ]:
transformers-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


SyntaxError: invalid syntax (ipython-input-2694070750.py, line 1)

In [ ]:
%cd /content/LLaMA-Factory
huggingface-cli upload --repo_id <your-username>/<model-name> --model_path ./content/LLaMA-Factory


SyntaxError: invalid syntax (ipython-input-1104885352.py, line 2)

In [ ]:
pip install transformers

In [ ]:
!transformers-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


2025-09-20 16:42:21.782756: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758386541.834875   46949 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758386541.850959   46949 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758386541.923698   46949 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758386541.923802   46949 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758386541.923813   46949 computation_placer.cc:177] computation placer alr

In [ ]:
!huggingface-cli login


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [ ]:
!python /content/LLaMA-Factory/src/train.py --config /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


2025-09-20 16:48:51.499571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758386931.548780   48519 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758386931.565844   48519 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758386931.605671   48519 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758386931.605741   48519 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758386931.605747   48519 computation_placer.cc:177] computation placer alr

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = torch_dtype
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [ ]:
finetuned_model_id = "/gdrive/MyDrive/finetune/finetune/llm-finetuning/models"
model.load_adapter(finetuned_model_id)

In [ ]:
def generate_resp(messages):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

response = generate_resp(translation_messages)